# Data Science Experience Hands On Lab1 - Python

The first thing we need to do is connect to a data source and read this into a spark sql dataframe.  Dataframes were new constructs starting in Spark 1.6 and provide a much richer symantex than the prior RDD approach.  

### Step 1 - Click on the "Find and Add Data" icon upper right (series of 1 & 0s) 

### Step 2 - In right panel under files, select "browse" and select the ACCIDENT2007-FullDataSet.csv file

Your File is locaed in project

### Next we will insert code into your notebook for that data file you loaded, so make sure your cusrson is in the next code cell below.

### Step 3 - Under file name, pick "Insert to code" > "Insert Spark SQL DataFrame" 

This will create a connection to the spark object store where the file is stored and create a resulting Spark SQL DataFrame for you.

In [ ]:
### Put your Cursor in this cell
### Do step 3 above
### Once code is pasted in here, Run the code in the cell


Now that you have a dataframe object, you can do analysis based on this such as performing correlation analysis on various variables

You can see the correlation between the individual was drunk and if the accident resulted in fatalities by performing a simple pearson correlation on the two variables.  The notebooks provide code assistance by using the tab after a dot notation to see the various functions availabe.

In [ ]:
### Run this code
df_data_1.corr('DRUNK_DR','FATALS','pearson')

DSX also provides the ability to import your favorite libraries to use within your notebooks, for example Pandas.  

Import the pandas library and use it to convert the Spark DataFrame to a Pandas DataFrame and use the head function to show the top 5 rows values.

In [ ]:
### Run this code
import pandas as pd
pd_fars = df_data_1.toPandas()
pd_fars.head()

You can also get summary statistics using the describe function.  This can help you determine missing values and the distribution of your attributes.

In [ ]:
### Run this code
pd_fars.describe()

Now we want to look at an individual states worth of data.  The Spark DataFrame object supports a filter option to allow you to filter the data based on a column of interest and the resulting value.  

To see a list of the states, the FARS data dictionairy provides a reference to the state codes here - https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812092 .  Choose your state of interest to filter the list down to your particular state.  Convert the results to a new Panda dataframe and view the first 5 rows data.  Below is a list of some of the state values;

        12 - Florida      11 - DC          13 - Georgia         48 - Texas    39 - Ohio     36 - New York        34 - New Jersey
        53 - Washingon    51 - Virginia    37 - North Carolina  40 - Oklahoma 32 - Neveda   45 - South Carolina  04 - Arizona

In [ ]:
### Run this code
df_cal=df_data_1.filter(df_data_1['STATE']==6)
pd_cal=df_cal.toPandas()
pd_cal.head(5)
pd_cal.count()

We would now like to map out the occurrences for the state of interest but if you look at the LATITUDE and LONGITUD values, these have not been declared as a float.

We can create new columns for a modified version of these fields so that we can map the individual occurances on a map.

Use lamba functions to create a lon and lat column that represents the expected values.

In [ ]:
### Run this codep
d_cal['lat'] = pd_cal['LATITUDE'].map(lambda x: (x * 1.0) / 1000000)
pd_cal['lon'] = pd_cal['LONGITUD'].map(lambda x: (x * -1.0) / 1000000)
pd_cal.head(5)

Now that we have the data we need, we can import the brunel package and use it to show a graphical map of the occurances

Use the following code to display the map for your state using the lon and lat values and use PERSONS to display a color scale based on the number of individuals involved in the accident.

In [ ]:
### Run this code
import brunel

%brunel map ('CA') + data('pd_cal') x(lon) y(lat) color(PERSONS) tooltip(VE_TOTAL)

Another graphics package that was created by IBM is the pixie dust package.  Pixie dust provides an easy way to visualize the data using various 
charts.  

To import the pixiedust package, you simply need to use and import statement (from pixiedust.display import *)

Once imported, you bring up the interactive display area by using the display function on your dataset - for example display(df_data_1).

The initial display is a table view of the dataframe.  

Switch views to the pie chart by selecting the middle charting drop down menu at the top left of the display area. This will display a pie chart of the count of accidents by state along with a percentage.  You can view and modify the options used for the display by selecting the paint brush icon at the bottom left of the display area (note this may be invisible until you hover near the area with your mouse).  If you change the value to city instead of state, you will see a busier graph.

You can switch to a different dataframe at anytime by changing the value in the display parameter and rerunning the cell.  

Change the display to use the california data display(df_cal).

select the histogram chart from the drop down and then modify the values to contain city.  

In [ ]:
### Run this code
from pixiedust.display import *
display(df_cal)

At this point we are ready to start our first pass at building a predictive model.  In this example we will use the statsmodel.formula.api package.  

In [ ]:
### Run this code
import statsmodels.formula.api as sm

result = sm.ols(formula="FATALS ~ VE_TOTAL + PERSONS + WEATHER + VE_FORMS", data=pd_cal).fit()
print result.params

To see a summary of all the results, use print result.summary().

As you can see from the results, this is not a good model at all.  Select a different set of attributes to see if you can improve the R-squared value.

In [ ]:
### Run this code
print result.summary()